# 基于Embedding的RNN生成唐詩

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
import pandas as pd
import random
import re

In [2]:
'''
'<GO_1>' => 五言絕句
'<GO_2>' => 五言律詩
'<GO_3>' => 七言絕句
'<GO_4>' => 七言律詩
使用不同的起始字，可以避免預測較長的詩句如五言律詩或七言律詩，在輸出時提前出現<EOS>
'''
special_words = ['<PAD>' , '<GO_1>' , '<GO_2>' , '<GO_3>' , '<GO_4>' , '<EOS>']
modes = ['五言絕句' , '五言律詩'  , '七言絕句' , '七言律詩']
poem_word_count = {'五言絕句' : 24 , '五言律詩' : 48 , '七言絕句' : 32 , '七言律詩' : 64}

# 讀取數據

In [3]:
# 分別讀取4個種類不同的詩集
data_number_distribution = [] 
sources , targets , poems = [] , [] , []

for i , mode in enumerate(modes):
    file_name = 'dataset/{}.txt'.format(mode)
    count = 0
    f = open(file_name , 'r' , encoding = 'utf-8')
    for j , line in enumerate(f.readlines()):
        try:
            title, content = line.strip().split(':')
            content = content.replace(' ' , '')
            content = content[:-1]
            content = content.replace('。' , '，') # 把'。'換成'，'應該會比較好訓練
            content += '。' 

            # 不符合字數或詞句中有不是中文&逗號&句號的都不考慮
            if len(content) == poem_word_count[mode] and re.search('[^\u4e00-\u9fa5。，]' , content) is None:
                source_temp = []
                for word in content:
                    source_temp.append(word)
                source = ['<GO_{}>'.format(i + 1)] + source_temp + ['<EOS>'] # 不同種類的詩句，起始字會不一樣
                target = source[2:] + ['<PAD>']
                sources.append(source)
                targets.append(target)
                poems.append(source_temp)
            else:
                if j % 50 == 0:
                    print('{} : {}'.format(mode , content))
            
        except ValueError:
            pass      
        
    data_number_distribution.append(len(sources))

五言絕句 : 一鴈過連營，繁霜覆古城，胡笳在何處？半夜起邊聲。
五言律詩 : 霜壓危簷冷，蕭條秋滿林，青雲知己惠，黄菊故園心，百慮憑祛遣，頹齡仗制禁，緬思彭澤叟，履運□懷深。
五言律詩 : 光風爲花好，奕奕弄清溫，撩理鶑情趣，留連蝶夢魂，飲酣浮倒暈，舞倦怯新翻，水竹傍□意，明紅似故園。
五言律詩 : 相隨萬里途，汝豈被名驅，掛壁疑何用，辭家不可無，店翁先洒掃，津吏認稱呼，舉子無錢刻，惟將□紙糊。
五言律詩 : 刻□□材圓，相成□不偏，守規心已正，受省禮仍全，有節通貞幹，無邪抱直弦，向非兼羽翼，何得札俱穿。
五言律詩 : 朝寒生廣陌，宿雨漲方池，霧柳和鴉重，風簾爲燕欹，□□□□□，□□□□□，多才惡摩詰，無日不成詩。
五言律詩 : 若以俗眼觀，榮枯幾翻手，若以法眼觀，窮達竟何有，吾身以覺贅，身外安足守，若□謝□□，爲我作不朽。
五言律詩 : 貧道我兒孝，姑云此婦賢，諸郎丹穴種，夫子玉堂仙，□□□寧忍，煎膠法不傳，多情潘騎省，應賦悼亡篇。
五言律詩 : 薌林蒼翠甚，極目可□詩，梅子風前落，杏花雨後移，清溪過半郭，孤月隠疏籬，寂寂雲山外，蕭然獨自知。
五言律詩 : 空庭風捲葉，壞廡雨淋碑，起廢須成厦，占孚預擊蓍，紅樓追曩集，碧落待誰詩，非有鈎牽□，□□亦未知。
五言律詩 : 俗流嗜喧卑，世才務高潔，築臺僅踰尋，清夜延明月，四垂天幕低，千里蟾光發，美人□□才，裁賦何清絕。
五言律詩 : 野人方蠖屈，天子正龍飛，東閣翹林騪，北山□□移，鵩來賈生去，熊兆太公歸，地主殷勤別，雲霄萬里期。
五言律詩 : 自憶專房寵，曾居第一流，移恩向何處？暫妬不容收，夜久絲管絕，月明宮殿秋，空將舊時意，長望鳳皇樓。
五言律詩 : 何地早芳菲？宛在長門殿，夭桃色若綬，穠李光如練，啼鳥弄花疎，遊蠭飲香徧，歎息春風起，飄零君不見。
五言律詩 : 白日供多病，青山且舊居，柴門臨水静，風葉舞霜餘，老練時情熟，貧□家計疏，牆東端可望，炙背飽翻書。
七言絕句 : 向來儲粟輔常平，招鶴翁曾爲證明，可是□□□□者，邑人久不識絃聲。
七言絕句 : 解用不須雙刃劍，延齡何必九還丹，尉遲□不在家死，獅子那能食兔錢。
七言絕句 : 迴樂峰前沙似雪，受降城外月如霜，不知何處吹蘆管？一夜征人盡望鄉。
七言絕句 : 何處春風吹曉幕？江南綠水通朱閣，美人二八面如花，泣向東風畏花落。
七言絕句 : 高高亭聳紫霄端，借榻高眠廛市間，輪

In [4]:
# 七言絕句與七言律詩的數據偏多
for i , mode in enumerate(modes):
    print(mode + ': {}'.format(data_number_distribution[i]))

五言絕句: 16903
五言律詩: 76755
七言絕句: 159515
七言律詩: 226746


# 數據預處理

In [5]:
# 統計每個字出現次數
all_words = []
for poem in poems:
    all_words += poem
    
# counter包含了每個字對應的頻率
counter = {}  
for word in all_words:
    if word not in counter.keys():
        counter[word] = 1
    else:
        counter[word] += 1
count_pairs = sorted(counter.items() , key = lambda x : x[1] , reverse = True) # 根據每個字出現的次數，由高排到低

# 取前多少個常用字，可以將len(words)換成其他數目，這裡是全部的字都收集起來
words = [i[0] for i in count_pairs]
words = special_words + words[:len(words) + 1]

In [6]:
# 每個字映射為一個數字ID
word_to_int = {}
for idx , word in enumerate(words):
    word_to_int[word] = idx
int_to_word = dict(zip(word_to_int.values() , word_to_int.keys()))

source_poem , target_poem = [] , []
for poem in sources:
    temp = []
    for word in poem:
        temp.append(word_to_int[word])
    source_poem.append(temp)
    
for poem in targets:
    temp = []
    for word in poem:
        temp.append(word_to_int[word])
    target_poem.append(temp)
    
# 將source_poem與target_poem中的順序全部打亂，每次訓練中NN都會看過'五言絕句' , '五言律詩' , '七言絕句' , '七言律詩'
shuffle_index = np.arange(len(source_poem))
np.random.shuffle(shuffle_index)
source_poem = list(np.array(source_poem)[shuffle_index])
target_poem = list(np.array(target_poem)[shuffle_index])

# 構建網絡

In [7]:
# 超參數
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 64
# RNN Size(RNN隱層結點數量)
rnn_hidden_unit = 128
# Embedding Dimension Size
embedding_size = 200
# Sequence Length
seq_length = 20
# layer numbers(總共2層RNN)
num_layers = 2 
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 50
# vocabulary size
vocab_size = len(word_to_int)

In [8]:
# 輸入層
input_data = tf.placeholder(tf.int32, [None , None])
targets = tf.placeholder(tf.int32 , [None , None])
on_train = tf.placeholder(tf.bool , name = 'on_train') # 決定train模式或者test模式

# Word Embedding

In [9]:
# 進行embedding
# input_data: 輸入的tensor
# vocab_size: 詞匯表大小
# embedding_size: 嵌入維度
embedding = tf.Variable(tf.random_uniform([vocab_size, rnn_hidden_unit], -1 , 1))
inputs = tf.nn.embedding_lookup(embedding , input_data)

# Build RNN

In [10]:
def get_lstm_cell(rnn_hidden_unit):
    lstm_cell = tf.contrib.rnn.LSTMCell(rnn_hidden_unit)
    return lstm_cell

cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_hidden_unit) for _ in range(num_layers)])

'''
在測試時，batch_size為1(只有1筆要測試的資料)，因此分為兩種不同輸入的情況
在測試的時候必須要用到init_state_for_test，因為再生成第2個之後的詞，都必須要將生成的last_State餵到init_state_for_test
'''
# output => [batch_size , ? , rnn_size] = [64 , ? , 128]
init_state_for_train = cell.zero_state(batch_size , dtype = tf.float32)
def train():
    output , last_state = tf.nn.dynamic_rnn(cell , 
                                            inputs , 
                                            initial_state = init_state_for_train , 
                                            dtype = tf.float32)
    return tf.identity(output) , tf.identity(last_state)

init_state_for_test = cell.zero_state(1 , dtype = tf.float32)
def test():
    output , last_state = tf.nn.dynamic_rnn(cell , 
                                            inputs , 
                                            initial_state = init_state_for_test , 
                                            dtype = tf.float32)
    return tf.identity(output) , tf.identity(last_state)  

output , last_state = tf.cond(on_train , train , test)
output = output[: , 1: , :] # 起始字'<GO_{}>對應的第一個RNN cell產生的output，不列入cross entropy的計算

# Build Neural Network(構建神經網絡，將RNN層與全連接層相連)

In [11]:
# [? , vocab_size]
output = tf.reshape(output , [-1 , rnn_hidden_unit])
weights = tf.Variable(tf.truncated_normal([rnn_hidden_unit , vocab_size]))
bias = tf.Variable(tf.zeros(shape = [vocab_size]))

logits = tf.nn.bias_add(tf.matmul(output , weights) , bias = bias)
# logits = tf.contrib.layers.fully_connected(output , vocab_size , activation_fn = None)

# 可以訓練的過程中看一下寫出什麼詩
predicting_logits = tf.nn.softmax(logits) 
predicting_logits_ = tf.argmax(predicting_logits , axis = 1)
predicting_logits_ = tf.reshape(predicting_logits_ , [batch_size , -1])

In [12]:
targets_ = tf.one_hot(tf.reshape(targets , [-1]) , depth = vocab_size)
loss = tf.nn.softmax_cross_entropy_with_logits(labels = targets_ , logits = logits)
total_loss = tf.reduce_mean(loss)

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
var_list = tf.trainable_variables()
gradients = tf.gradients(total_loss , var_list)
# 利用gradient clippling的方式來防止梯度爆炸
# 即通過設置一個threshold，當gradients超過這個threshold時，就將gradients變為threshold
# 這就保證了梯度不會變得很大
capped_gradients = [(tf.clip_by_value(grad, -1. , 1.)) for grad in gradients if grad is not None]      
train_op = optimizer.apply_gradients(zip(capped_gradients , var_list))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



# 構造batch

在機器人寫唐詩這個主題中，source與target分別如下:
    
    source :  <GO_{}>  得  不  到  的  就  更  加  愛   ,    太   容  易  來  的  就  不  理   睬    <EOS>
    target :           不  到  的  就  更  加  愛   ,  太    容   易  來  的  就  不  理  睬  <EOS>  <PAD>
     
其實target就是將source往右移動一格，達到錯位的效果<br>
PS : `<GO>`與`<EOS>`不見得都需要加

In [13]:
def get_batches(batch_i):    
    start_index = batch_i * batch_size
    end_index = start_index + batch_size
    source_batch = source_poem[start_index : end_index].copy()
    target_batch = target_poem[start_index : end_index].copy()
    
    # 找到這個batch的所有poem中最長的poem的長度  
    max_source_length = max([len(batch) for batch in source_batch]) 
    max_target_length = max([len(batch) for batch in target_batch])
    
    # 將batches中的每個詩句的長度都填充到與max_length一樣長，填充值為word_to_int['<PAD>']
    source_batch_pad , target_batch_pad = [] , []
    source_length , target_length = [] , []
    for i in range(0 , batch_size):
        source_length.append(len(source_batch[i]))
        target_length.append(len(target_batch[i]))
        
        source_temp = source_batch[i].copy()
        while len(source_temp) < max_source_length:
            source_temp.append(word_to_int['<PAD>'])
            
        target_temp = target_batch[i].copy()    
        while len(target_temp) < max_target_length:
            target_temp.append(word_to_int['<PAD>'])
            
        source_batch_pad.append(source_temp)
        target_batch_pad.append(target_temp)
        
    source_batch_pad = np.array(source_batch_pad)
    target_batch_pad = np.array(target_batch_pad)    
    source_length = np.array(source_length)
    target_length = np.array(target_length)        
    """
    source_batch_pad : [<GO> ,  從 , 前 , 明 , 月 ,  光   ,  <EOS>  ,  <PAD>   ,  <PAD>]
    target_batch_pad : [        前 , 明 , 月 , 光 , <EOS> ,  <PAD>  ,  <PAD>   ,  <PAD>]          
    source_batch_pad : [<GO> ,  月 , 落 , 烏 , 啼 ,  霜   ,   滿    ,   天     ,  <EOS>]
    target_batch_pad : [        落 , 烏 , 啼 , 霜 ,  滿   ,   天    ,  <EOS>   ,  <PAD>] 
    """    
    return source_batch_pad , target_batch_pad , source_length , target_length

# 模型訓練

In [14]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

n_chunk = len(source_poem) // batch_size
stop_early = 0
loss_ = []
for epoch_i in range(0 , num_epochs):
    loss_batch = []
    for batch_i in range(0 , n_chunk):
        source_batch_pad , target_batch_pad , _ , target_length = get_batches(batch_i)
        
        feed_dict = {input_data : source_batch_pad ,
                     targets : target_batch_pad ,  
                     on_train : True}
                             
        _ , loss = sess.run([train_op , total_loss] , feed_dict)
   
        # 每一個階段就來看一下結果
        if batch_i % show_every_n_batches == 0:
            index = random.randint(0 , batch_size - 1) # 隨機從 0 ~ batch_size-1 選取1個index來觀看預測出來的詩句
            source_value, target_value , predicting_value = sess.run([input_data , targets , predicting_logits_ ] , feed_dict)
            
            source_value = source_value[index , :]
            target_value = target_value[index , :]
            predicting_value = predicting_value[index , :]
            
            source_result = ''
            for int_source in source_value:
                if int_to_word[int_source] == '<EOS>' : break
                source_result += int_to_word[int_source]
                
            target_result = ''
            for int_target in target_value:
                if int_to_word[int_target] == '<EOS>' : break
                target_result += int_to_word[int_target]
            
            predicting_result = ''
            for int_predicting in predicting_value:
                if int_to_word[int_predicting] == '<PAD>' : break
                predicting_result += int_to_word[int_predicting]

            print('=' * 90)    
            print('source_value : {}'.format(source_result))
            print('~' * 80)
            print('target_value : {}'.format(target_result))
            print('~' * 80)
            print('predicting_value : {}'.format(predicting_result))
            print('~' * 80)
            print('epoch_i : {}'.format(epoch_i))
            print('batch_i : {}'.format(batch_i))
            print('loss : {:.4f}\n'.format(loss))
            
        loss_batch.append(loss)
    
    loss_batch = np.array(loss_batch)  
    loss_mean = loss_batch.mean()  
    print('*' * 90)
    print('batch_loss_mean : {:.4f}'.format(loss_mean))
    print('stop_early : {}'.format(stop_early))
    
    # 隨著訓練的演進所存下的model會有不同的風格
    if epoch_i % 6 == 0 and epoch_i > 0:
        saver.save(sess , './model/' , global_step = epoch_i)
    
    if epoch_i == 0:
        loss_.append(loss_mean)
    elif epoch_i > 0:    
        loss_ewm = pd.Series(loss_)
        loss_ewm = loss_ewm.ewm(alpha = 0.7 , adjust = False).mean()
        if loss_mean <= loss_ewm.iloc[-1]:
            print('New Record!\n')
        else: 
            print('No Improvement.\n')
            stop_early += 1
            if stop_early == 10: break 
        loss_ = list(loss_)  
        loss_.append(loss_mean)   

source_value : <GO_4>嬌朱淺淺透冰光，瘦倚疏篁半出牆，雅有風情勝桃杏，巧含春思避冰霜，融明醉臉籠輕暈，斂掩仙裙蹙嫩黄，日暮風英墮行袂，依稀如著領巾香。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 朱淺淺透冰光，瘦倚疏篁半出牆，雅有風情勝桃杏，巧含春思避冰霜，融明醉臉籠輕暈，斂掩仙裙蹙嫩黄，日暮風英墮行袂，依稀如著領巾香。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 0
loss : 9.0544

source_value : <GO_3>碧天如水水明霞，雁字橫風一一斜，袖手行吟不知晚，滿川霜月浸蘆花。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 天如水水明霞，雁字橫風一一斜，袖手行吟不知晚，滿川霜月浸蘆花。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 風風風，，，，不不不，，，，，不人不人，，，，一人風不，，，，<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 50
loss : 4.2449

source_value : <GO_4>太極虚空亦有根，打開闗捩是何人，丹田旋轉乾坤槖，碧海循環日月輪，久矣我前曾有我，悠哉身外豈無身，分明好個通天竅，只要時人認得真。
~~~~~~~

source_value : <GO_3>壯士縛雞聊爾耳，小兒相鼎竟何如，眼前萬事不堪說，惟有秋風解得渠。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 士縛雞聊爾耳，小兒相鼎竟何如，眼前萬事不堪說，惟有秋風解得渠。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 年無春無可事，一詩無與不何如，一前不里無能問，一有東風一笑詩。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 750
loss : 3.5109

source_value : <GO_1>白雲如高人，擇友尚傲世，朱門客三千，一點不可致。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 雲如高人，擇友尚傲世，朱門客三千，一點不可致。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 髮無水水，山地不相塵，不門不不月，何笑一可得。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 800
loss : 3.2242

source_value : <GO_2>翠萼凌晨綻，清香逐處飄，高低臨曲檻，紅白間纖條，潤比攢溫玉，繁如簇絳綃，盡堪圖畫取，名筆在僧繇。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

source_value : <GO_4>風定舟行力不難，一鈎新月照溪灣，斷烟橫抹岸頭樹，落日斜欹江上山，魚鳥久知思舊侶，桑榆歸去亦慙顔，兒童見我應相怪，兩鬢如何雪樣斑。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 定舟行力不難，一鈎新月照溪灣，斷烟橫抹岸頭樹，落日斜欹江上山，魚鳥久知思舊侶，桑榆歸去亦慙顔，兒童見我應相怪，兩鬢如何雪樣斑。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 流山行不未知，一年風月照春邊，一雲風水雲邊水，一日風陽水上春，一躍不來何舊事，山麻不去不無無，一童不說無無憶，不鬢無飛不未新。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 1450
loss : 3.4027

source_value : <GO_2>今日寢門側，涕流予不堪，伊人竟長逝，回使返空函，如覺知音絕，深懷掛劍慚，仍聞身後事，生女不生男。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 日寢門側，涕流予不堪，伊人竟長逝，回使返空函，如覺知音絕，深懷掛劍慚，仍聞身後事，生女不生男。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 日無門路，今零無未知，山川無何在，一首一江城，一今無何在，何居不石扉，不聞一在事，不死不知時。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
ba

source_value : <GO_2>舊憶陵陽北，林園近板橋，江晴帆影滿，野迥鶴聲遙，鳥徑通山市，汀扉上海潮，秦城歸去夢，夜夜到漁樵。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 憶陵陽北，林園近板橋，江晴帆影滿，野迥鶴聲遙，鳥徑通山市，汀扉上海潮，秦城歸去夢，夜夜到漁樵。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 雨西山路，相深日水橋，山山山影遠，山水落飛喧，野語秋秋色，山洲水月邊，何人無未路，何雨夜西舟。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 2200
loss : 3.3780

source_value : <GO_1>近知名阿侯，住處小江流，腰細不勝舞，眉長惟是愁。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 知名阿侯，住處小江流，腰細不勝舞，眉長惟是愁。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 日無姓子，不處不山南，不間不可地，不頭不有渠。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 2250
loss : 3.3980

source_value : <GO_1>魄依鉤樣小，扇逐漢機團，細影將圓質，人間幾處看。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

source_value : <GO_4>熱惱煩蒸正不禁，火輪忽向半空沈，屯中信有雲雷象，復處方知天地心，便擬新秋先上印，何須晚歲始投簪，江山信美非吾土，栗里時從蝶夢尋。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 惱煩蒸正不禁，火輪忽向半空沈，屯中信有雲雷象，復處方知天地心，便擬新秋先上印，何須晚歲始投簪，江山信美非吾土，栗里時從蝶夢尋。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 事人君不自休，不雲猶忽白天春，風雲不是無中去，老見無知水地來，風是相詩供作客，不妨一節作知家，誰湖不是無無事，不里無時一夢回。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
batch_i : 2950
loss : 3.5188

source_value : <GO_2>半夜沉清影，宵殘猶伴吟，已甘垂樹杪，由得掛天心，淡色迷蓬戶，餘光映宿禽，誰家傷早別，凝思共沉沉。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 夜沉清影，宵殘猶伴吟，已甘垂樹杪，由得掛天心，淡色迷蓬戶，餘光映宿禽，誰家傷早別，凝思共沉沉。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 夜秋沉夜，清寒夜未人，風聞春白杪，不是一衣風，野月無山島，清香滿水時，不知一遠晚，不水不徘冥。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 0
ba

...

source_value : <GO_3>散髪南風書曲肱，衰疲真自笑無能，劇談稍負茅簷月，强賦荆州學少陵。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 髪南風書曲肱，衰疲真自笑無能，劇談稍負茅簷月，强賦荆州學少陵。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 髮無山不卷曲，一年不欲是人心，不談不覺清簷語，不與詩溪一釣安。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 11
batch_i : 3250
loss : 3.2556

source_value : <GO_4>忽被黄符迫下山，許携苕帚面天顔，早知錦纜盛宮女，因舍紅巾問阿環，空有蚋蠅侵玉骨，斷無龍虎守金丹，秋墳翁仲相酬答，日落僧歸家閉關。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 被黄符迫下山，許携苕帚面天顔，早知錦纜盛宮女，因舍紅巾問阿環，空有蚋蠅侵玉骨，斷無龍虎守金丹，秋墳翁仲相酬答，日落僧歸家閉關。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 忽春花不雨車，不來一霅出平開，山知老繡江相闕，不作金花入舊顔，一使玉絲隨玉麈，不無風頷照天丹，不風不媼無如酢，不日西房不有關。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 11
batch_i : 3300
loss : 3.0392

source_value : <GO

source_value : <GO_2>嶄然頭角異，年少銳功名，身爲才高累，名因宦謫成，山川助奇氣，騷雅變遺聲，千古羅池祀，昌黎報友情。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 然頭角異，年少銳功名，身爲才高累，名因宦謫成，山川助奇氣，騷雅變遺聲，千古羅池祀，昌黎報友情。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 然無上柳，不少不心名，不世人難志，心高世有難，不川無清絕，天雅在天音，不古無衣上，何黎不此聲。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 12
batch_i : 450
loss : 3.3090

source_value : <GO_4>文書堆案眼慵開，客有可人今雨來，說劍丹心空耿耿，岸巾雪鬢欲皚皚，山田漸喜滋香稻，客位從教長紫苔，涸轍窮鱗頼濡沫，未宜曷月詠歸哉。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 書堆案眼慵開，客有可人今雨來，說劍丹心空耿耿，岸巾雪鬢欲皚皚，山田漸喜滋香稻，客位從教長紫苔，涸轍窮鱗頼濡沫，未宜曷月詠歸哉。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 章不案不中開，一子詩憐無日來，一得不青無自耿，故巾飛白已成皚，山川不覺無新味，野子無容白樂苔，莫澤不途何吾墨，不妨一渴到江來。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 12
b

source_value : <GO_3>春入風流別乘家，滿園桃李掞春華，殷勤爲我傾醇酎，要看庭前百樹花。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 入風流別乘家，滿園桃李掞春華，殷勤爲我傾醇酎，要看庭前百樹花。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 風春流不有春，一城春李又芳花，不勤一我歸杯酒，莫使春前一尺花。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 12
batch_i : 1200
loss : 2.8105

source_value : <GO_1>日向陰虬永，春隨綵鳳歸，玉京多燕喜，金殿滿光輝。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 向陰虬永，春隨綵鳳歸，玉京多燕喜，金殿滿光輝。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 月山雲秀，人來水仗飛，春堂無少子，不玉不庭輝。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 12
batch_i : 1250
loss : 3.0182

source_value : <GO_4>君貧賣術我賣文，君貧似我貧一分，君挾天盤走湖海，我携破硯登青雲，兩窮相值君莫笑，賣文有時饑可療，天邊一點少微星，會與太陽來合照。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

source_value : <GO_2>昔在西都日，趨庭見伯魚，金華同勸講，石室共紬書，鮑叔深知我，顔淵實喪予，衰年哭心友，忍復望靈車。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 在西都日，趨庭見伯魚，金華同勸講，石室共紬書，鮑叔深知我，顔淵實喪予，衰年哭心友，忍復望靈車。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 日西湖去，今朝一玉夷，天閨三一我，玉窌不誰書，日叔今藏己，詩公不不家，何翁無不事，何看問鄉槎。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 12
batch_i : 1950
loss : 2.9197

source_value : <GO_4>冷烟無計了愁端，花滿湖陰百頃寬，渺邈家山身獨老，周迴園館眼長寒，吹餳近市徒誇賣，上冢何人不鮮歡，空對海棠春睡足，數枝紅濕露漙漙。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
target_value : 烟無計了愁端，花滿湖陰百頃寬，渺邈家山身獨老，周迴園館眼長寒，吹餳近市徒誇賣，上冢何人不鮮歡，空對海棠春睡足，數枝紅濕露漙漙。
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
predicting_value : 落深路似無人，一落江邊水里寬，一渺不山無不健，老旋風圃不猶青，風花酒日無勞我，醉馬無妨更見明，不有清棠春事起，不聲紅杏更花漙。<EOS>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch_i : 12


KeyboardInterrupt: 

In [15]:
'''
在這裡產生詩句的方法 :

iter 1:
begin_word = [<GO_1 , '花']，丟到訓練好的RNN cell中，取最後一個RNN cell產生的字['開']

iter 2:
begin_word = [<GO_1 , '花' , '開']，丟到訓練好的RNN cell中，取最後一個RNN cell產生的字['花']

iter 3:
begin_word = [<GO_1 , '花' , '開' , '花']，丟到訓練好的RNN cell中，取最後一個RNN cell產生的字['落']

iter 3:
begin_word = [<GO_1 , '花' , '開' , '花' , '落']，丟到訓練好的RNN cell中，取最後一個RNN cell產生的字['落']
'''
def compose_poem(start_token , start_word):
    # 定義啟動的單詞
    begin_word = [word_to_int['<GO_{}>'.format(start_token)] , word_to_int[start_word]]
    begin_input = np.full([1 , len(begin_word)] , begin_word)
    gen_sentences = ['<GO_{}>'.format(start_token) , start_word]
    word = ''
    
    time_step = 0
    while word != '<EOS>':
        state_current = sess.run(init_state_for_test)
    
        feed_dict = {input_data : begin_input ,
                     on_train : False,
                     init_state_for_test : state_current}
        
        prob = sess.run(tf.reshape(predicting_logits , [1 , time_step + 1 , vocab_size]) , feed_dict)
        time_step += 1
        prob = prob[: , -1 , :][0]

        # 以所有詞的機率分布選出下一個詞，在這裡先以機率最大的那個詞先當作下一個詞
        # 但也可以裡用這個機率分布sample出下一個詞，這樣每次產生的詩會比較多元
        int_ = prob.argmax()
        word = int_to_word[int_]
    
        begin_word.append(word_to_int[word])
        begin_input = np.full([1 , len(begin_word)] , begin_word)
        gen_sentences.append(word)
     
    print('=' * 90)
    print(''.join(gen_sentences))

compose_poem(1 , '花')
compose_poem(2 , '花')
compose_poem(3 , '花')
compose_poem(4 , '花')

<GO_1>花開花滿眼，花發春來春，春風不可見，春色不可攀。<EOS>
<GO_2>花下春風裏，春風雨後晴，春風初日暖，春色已春晴，春色春風暖，春風柳絮催，春風一杯酒，不待醉中看。<EOS>
<GO_3>花開花落不知春，春色春風不可人，不是春風吹不盡，不知春色滿花枝。<EOS>
<GO_4>花開花落不成春，春色春風不可移，春色已隨春色好，春風初過柳花新，春風不動春風起，春草無人綠滿枝，不是春風吹不得，不知春色滿花枝。<EOS>


In [16]:
compose_poem(1 , '劉')
compose_poem(2 , '劉')
compose_poem(3 , '劉')
compose_poem(4 , '劉')
compose_poem(1 , '子')
compose_poem(2 , '子')
compose_poem(3 , '子')
compose_poem(4 , '子')
compose_poem(1 , '寧')
compose_poem(2 , '寧')
compose_poem(3 , '寧')
compose_poem(4 , '寧')

<GO_1>劉郎不可見，不是不可知，不知無所得，不是不可知。<EOS>
<GO_2>劉郎不可見，不復見人同，不是人間事，何如我自知，不知心事少，不是世人知，不是人間事，何人與我歸。<EOS>
<GO_3>劉郎不是一枝花，一片春風不肯留，不是春風吹不盡，不知春色滿花花。<EOS>
<GO_4>劉郎不是一番春，一片春風不可人，一片春風無限意，一枝花裏雪如絲，春風不作春風起，春色無人細細吹，不是春風吹不得，不知春色滿花枝。<EOS>
<GO_1>子子不可見，不知不可知，不知無所得，不是是非非。<EOS>
<GO_2>子美不可見，不復爲君同，不是人間事，何須不可論，不知三世事，不是一生非，不是人間事，何人與我同。<EOS>
<GO_3>子規不用上天真，一片春風不可知，不是一枝無一點，不知何處是春風。<EOS>
<GO_4>子規一點一枝香，一片春風一夜凉，一片春風吹不動，一枝風雨不成寒，春風不動春風起，春色無人雨後寒，不是春風吹不盡，不知春色滿花開。<EOS>
<GO_1>寧有一身事，不知心事空，不知無一事，不是是非非。<EOS>
<GO_2>寧論一生死，不復爲吾衰，不是一生死，不知三百年，不知無所事，不是不成身，不是無人識，何人不可求。<EOS>
<GO_3>寧知一日一番春，不是春風一樣春，不是春風吹不盡，不知春色滿花枝。<EOS>
<GO_4>寧知一日一番春，不見春風一日春，春色已隨春色好，春風初過柳花新，春風不動春風起，春草無人綠滿池，莫笑春風吹不盡，不知春色滿江村。<EOS>


In [17]:
compose_poem(1 , '洪')
compose_poem(2 , '洪')
compose_poem(3 , '洪')
compose_poem(4 , '洪')
compose_poem(1 , '國')
compose_poem(2 , '國')
compose_poem(3 , '國')
compose_poem(4 , '國')
compose_poem(1 , '彭')
compose_poem(2 , '彭')
compose_poem(3 , '彭')
compose_poem(4 , '彭')

<GO_1>洪崖一萬仞，一片一峰青，不知無一事，不是是非心。<EOS>
<GO_2>洪範無人見，無人不可知，不知三世事，不是一生身，不是人間事，何須事往時，不知無處問，不是是非非。<EOS>
<GO_3>洪崖一點一生真，一片雲山一點春，不是一身無一事，不知何處是人人。<EOS>
<GO_4>洪崖一日一番春，一片春風一夜春，一片春風無限意，一枝花發雪初飛，春風不動春風起，春色無人細雨多，不是春風吹不得，不知春色滿花花。<EOS>
<GO_1>國士不可見，不知何所之，不知無所得，不是不能知。<EOS>
<GO_2>國事無人識，何人得自由，不知三世事，不是一生無，不是人間事，何須事往休，不知無此事，不是是非無。<EOS>
<GO_3>國事無人不可論，不知何處是何人，不知不是無人識，不是人間不可知。<EOS>
<GO_4>國士無人不可攀，一身不復到天台，一身不是三千界，萬里無人一片心，一片一聲無一點，一聲寒雨夜深深，不知不是無人見，不是人間不是心。<EOS>
<GO_1>彭澤無人見，山中不可攀，不知無處問，不是是非心。<EOS>
<GO_2>彭澤三千里，江湖一水濱，山深山色好，水落水雲新，野水連雲合，山林落日斜，此心無一事，不是是非無。<EOS>
<GO_3>彭澤西風一夜寒，一枝紅葉滿江城，不知不是春風裏，不是春風一樣來。<EOS>
<GO_4>彭澤西風一夜寒，江南春色滿江城，春風一夜春風起，春草萋萋草木深，一片春風吹不斷，一枝風雨夜無聲，不知風雨無人見，不是春風一夜來。<EOS>


In [18]:
def compose_poem_(start_token , start_word):
    # 定義啟動的單詞
    begin_word = [word_to_int['<GO_{}>'.format(start_token)] , word_to_int[start_word]]
    begin_input = np.full([1 , len(begin_word)] , begin_word)
    gen_sentences = ['<GO_{}>'.format(start_token) , start_word]
    word = ''
    
    time_step = 0
    while word != '<EOS>':
        state_current = sess.run(init_state_for_test)
    
        feed_dict = {input_data : begin_input ,
                     on_train : False,
                     init_state_for_test : state_current}
        
        prob = sess.run(tf.reshape(predicting_logits , [1 , time_step + 1 , vocab_size]) , feed_dict)
        time_step += 1
        prob = prob[: , -1 , :][0]

  
        # 以所有詞的機率分布sample出下一個詞，但不是挑出機率最大的那個詞當作下一個詞
        int_ = np.random.choice(len(prob) , 1 , p = prob)
        word = int_to_word[int_[0]]
    
        begin_word.append(word_to_int[word])
        begin_input = np.full([1 , len(begin_word)] , begin_word)
        gen_sentences.append(word)
     
    print('=' * 90)
    print(''.join(gen_sentences))

compose_poem_(1 , '花')
compose_poem_(2 , '花')
compose_poem_(3 , '花')
compose_poem_(4 , '花')

<GO_1>花下兩三芳，遠橫楊家春，時復別番城，明月對東風。<EOS>
<GO_2>花裏境茫茫，帆生城北人，客來歸故舊，天竟掃除塵，常引詩書祗，還聞說夢新，年時俱卜築，松下聳城門。<EOS>
<GO_3>花開宮殿不禁柳，惠馥相清條異蕊，姮娥從睡美人嬌，誰知去去將妾恨，要喫新詩證一枝。<EOS>
<GO_4>花時故國徹無時，陌上歸來事簡敦，舉世歸來長者少，數時權上大三旬，命情故吏西峯客，礧磈嵯峨爛熳歌，好把青門遺世事，明當佳子范雎權。<EOS>


In [19]:
compose_poem_(1 , '劉')
compose_poem_(2 , '劉')
compose_poem_(3 , '劉')
compose_poem_(4 , '劉')
compose_poem_(1 , '子')
compose_poem_(2 , '子')
compose_poem_(3 , '子')
compose_poem_(4 , '子')
compose_poem_(1 , '寧')
compose_poem_(2 , '寧')
compose_poem_(3 , '寧')
compose_poem_(4 , '寧')

<GO_1>劉盛柏柏來，一鶚御恩深，非才五讓者，心空與重登。<EOS>
<GO_2>劉君持脩竹，便絕登高望，王孫幾解組，龍蛇勞洗沫，別雲東閣飛，東西西郭王，釣墮蘆荻洲，涉川容照目。<EOS>
<GO_3>劉公比我蔭重來，濃透煙泉晝繡沈，千古天涯音笑後，随風歸去鬢如尋。<EOS>
<GO_4>劉郎龜土漢戎題，跨鳳芒鞋八萬峰，下壘釣璜留騎上，烟花近去禁阡橋，歌風舞壓祠臺角，松菊蕭條暗樹圍，兜率窮山立晴露，壽封寬奉往辭帷。<EOS>
<GO_1>子君聞築臺，誰鳴種竹苓，顧外欲飛來，是久去歸程。<EOS>
<GO_2>子産威專路，寒更出却披，風檣依舊處，燕地寫長松，別鵠那堪醉，旌弦定正狂，濤洲青淨榻，時節賦清文。<EOS>
<GO_3>子虛曾爲偈元題，乞得真儒日俊髦，飽食不能生涸飲，終然周遍水長鱗。<EOS>
<GO_4>子中父子病將殘，應被臨風著法翰，林下疏梅珍品笑，山僧無數路縈欒，弓車元有無中甚，鏤鏡橫回破後災，白玉一同三戶寺，指甌詩在里閭明。<EOS>
<GO_1>寧是靈雲直，長安水鼠魂，參軍大奇獸，寧遣別田王。<EOS>
<GO_2>寧念長悠颺，飛梭縫五莖，蛙枝多似猛，衣食竟無媒，敗作烏龍去，鞭孫定有尋，坦哉攝古見，朝隕報再來。<EOS>
<GO_3>寧容斗柄歲華別，但擬宮姬入菊香，憐恨歸途不知處，事生早晚遇吾君。<EOS>
<GO_4>寧論野裏龐公去，百十門前合病人，履住柴荆華井社，雲疎菡萏白雲身，明朝塞上春無雨，駐舌家家綠漲迷，却喜離騷照絲扇，欲調短句總三更。<EOS>


In [20]:
compose_poem_(1 , '洪')
compose_poem_(2 , '洪')
compose_poem_(3 , '洪')
compose_poem_(4 , '洪')
compose_poem_(1 , '國')
compose_poem_(2 , '國')
compose_poem_(3 , '國')
compose_poem_(4 , '國')
compose_poem_(1 , '彭')
compose_poem_(2 , '彭')
compose_poem_(3 , '彭')
compose_poem_(4 , '彭')

<GO_1>洪鄭萬年恨，非關寸已萌，不憂文作死，安得欲投閑。<EOS>
<GO_2>洪岡往陰岑，歡遠渺無疆，遇說該未滿，長安每乞中，靈陰祠背色，濁酒志茫茫，唯有行年友，題歌寄此情。<EOS>
<GO_3>洪波久矣繄華枝，寸素青衫覓短衣，争把金樽開浣首，要須盜害易鳴嘶。<EOS>
<GO_4>洪崖高臥石榴花，蝶伴高迴蜀水流，側架座外佳句許，狂雲開鉢放舟疏，諸峰爲客貂裘滅，長嘯深愁只自憐，此夕相逢卻幽興，依然峰頂自隨緣。<EOS>
<GO_1>國學今年事，古來高作逢，相思邊已息，此意不那知。<EOS>
<GO_2>國學且招人，甘稱免若涯，白雲居重遠，先賦出他年，衰髮愁雍恨，驚鴻潔頌非，早爲千騎遠，埋日且酣歌。<EOS>
<GO_3>國在宦途仍起愁，古今投劍皖山川，日長哀怨愁初死，綠布風霜阻一漁。<EOS>
<GO_4>國身縛空餘寂言，間日前來百里邊，忠孝昔賢緣習義，奪車終合憶諸賢，黛絲已入湘闈路，桑葉重輕幾節車，餐芋抽貂三獻印，肯陪鍾室入前頭。<EOS>
<GO_1>彭澤笙何處，娘杯長語歌，憐君能路騎，苦客正同誇。<EOS>
<GO_2>彭澤晉荆州，廟前多郭邊，蕭屯迎日睡，詠甲固推年，乃慎論仁道，哀矜螗鼓瑕，淮平非怨志，視累咽滄洲。<EOS>
<GO_3>彭林州上步如箝，馬免丹成底已休，世事如親窮著筆，試看藤笠尚須臾。<EOS>
<GO_4>彭王峯上卧林藪，潘帳經年禾灌田，敗瓦周顔日霜巷，黃羊乳燕小龍眠，燕依殘日花塵在，藕色長箋賣酒家，歲晚山岑畏蒸節，裴回葉上女吟鬚。<EOS>


In [21]:
compose_poem_(1 , '廖')
compose_poem_(2 , '廖')
compose_poem_(3 , '廖')
compose_poem_(4 , '廖')
compose_poem_(1 , '素')
compose_poem_(2 , '素')
compose_poem_(3 , '素')
compose_poem_(4 , '素')
compose_poem_(1 , '月')
compose_poem_(2 , '月')
compose_poem_(3 , '月')
compose_poem_(4 , '月')

<GO_1>廖出彌移出，欽身竟可能，能文無競補，材大本本來。<EOS>
<GO_2>廖禍章康運，重重理亦殊，不妨才上月，猶笑世塵身，病苦辭吾黨，詩咏不任傳，新雪亦成閣，榮甫污吾州。<EOS>
<GO_3>廖王剖使謫眉論，三歲升車副徙途，出澗陽蛇論一色，此身自請典平錢。<EOS>
<GO_4>廖祿餘年自苦速，春風不忍寄蘇良，已開投地秦濤壟，不害南圍水墨雲，劒內分暉成事事，雲留古塚逐風濤，因經四分詩如遠，說爲長年徒自知。<EOS>
<GO_1>素幼無欺易，花葩只瑩冰，公來揚馬在，曾是一驪融。<EOS>
<GO_2>素日柴荆陌，嬌鶯囀子齊，暗香君未斷，錦縞鬢成絲，瑞水伊浮脆，空分國際春，時攜車馬發，不自待青春。<EOS>
<GO_3>素風低處雨天低，夢去誰知物外身，背送紅爐花口有，蟹搊拈出更難伸。<EOS>
<GO_4>素性全昌一日偏，旋春終日有餘村，俗須陶作無巾折，百畝拳成面樣根，居士不逢今白日，却吟重見入明秋，無燈佛塔肝腸熟，差覺長途一笑多。<EOS>
<GO_1>月勝光明眼，蘧降一粒紅，空恨未隨曲，不以莫惜時。<EOS>
<GO_2>月旦風標滅，天涯盡處風，軻生塵上塔，渡口瀫江鄉，尚有江邊路，惟留水際香，自慙離是客，相憶寄公懷。<EOS>
<GO_3>月上松梁一一家，如何入掃入莓天，箇中一事猶如畫，此寺能忘王父家。<EOS>
<GO_4>月上香來沐藥開，近鄰渾在意閑閑，来還西管無佳句，流引青帆影以生，遠近金丸無處解，高鳴冷竹遠燒來，南陽無主浮遊老，依舊重來造物榮。<EOS>


In [22]:
compose_poem_(1 , '洪')
compose_poem_(2 , '洪')
compose_poem_(3 , '洪')
compose_poem_(4 , '洪')
compose_poem_(1 , '瑞')
compose_poem_(2 , '瑞')
compose_poem_(3 , '瑞')
compose_poem_(4 , '瑞')
compose_poem_(1 , '堂')
compose_poem_(2 , '堂')
compose_poem_(3 , '堂')
compose_poem_(4 , '堂')

<GO_1>洪鬰寺山上，舟開眼界斜，門前無數約，坐久日西游。<EOS>
<GO_2>洪嶺聘澤廟，銘衣振馬征，星筒閨殿闕，河凍楚江東，病客休神沐，曾寒故國心，愧生寒尚想，銀筆寄人功。<EOS>
<GO_3>洪崖綠沒風酸緊，風葉擎來襲欲窮，西第空山霑縹塊，碧雲蒼翠值清風。<EOS>
<GO_4>洪岡匹馬契中仙，幾夜臨川世士頭，耐捷只憑雙鬢髮，帝心直在一枝秋，爽風雨冷人閑底，悲曉星花戀不收，準擬流光相對舉，形容接畛覆成秋。<EOS>
<GO_1>瑞草千絲立，囊巖萬竅齊，靈臺跨仙暇，聊以隔遺蹤。<EOS>
<GO_2>瑞雨郊原遠，田園便愛苔，綠房偏日晚，山水沓游行，爽氣色中桂，江梅花駐蘭，市貧堪拾取，那厭世塵催。<EOS>
<GO_3>瑞草荒蕪日正新，恩高衝霧罥帆飛，便從此地更空鏁，千幅橋邊不識真。<EOS>
<GO_4>瑞袖會西心辟離，東風到客醉新詩，朱衫可得能攏絆，青錦含來日再高，綠水曉光垂釣練，黄梅忽在水聲遲，江頭不信南州動，天下須容繡履風。<EOS>
<GO_1>堂上四禪竹，故應真是癡，關幽懸一倒，無復覔時僧。<EOS>
<GO_2>堂上棟梁盡，澄閻柱漢聲，弗能穿翠幄，可值濾中長，素翰輟良釋，聞推好儉咨，海門多折角，一古不思香。<EOS>
<GO_3>堂堂自古何人誦，仙杏經年取次詩，暇耄謾誇髭面節，絕憐逋裏不嫌人。<EOS>
<GO_4>堂上松篁古壁斑，長江無興見湘城，性情到樹臨秋色，天外廣陽顔愈深，後試仙心追孺子，但隨水鳥寄天寧，骨機閑坐孤心許，水落遙冤莫挽回。<EOS>
